# Workflow #4

This notebook runs Priority Score assessments.

In [ ]:
import algo.net_helper as nh
import algo.helper as h 
import os.path
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt


# settings
aoi_names = ["at_zs", "at_wien", "at_ib", "at_no", "at_zw", "at_graz_15"]

centrality_variant = "sbc_c4000_bp_d4"

file_suffix = ""

dir_data = "data"

mode = "bike_incwalk"
# tolerable access is determined by input network: all segments that have an index value assigned 
# (other than NULL, > 0) but have mode access set to False
access = "bicycle" 

# generated params
index_col_ft = f"index_{mode}_ft"
index_col_tf = f"index_{mode}_tf"
file = os.path.join(dir_data, f"r_<aoi_name>_edges{file_suffix}.gpkg")
file_ps = os.path.join(dir_data, f"ps_<aoi_name>{file_suffix}.gpkg")
ccol_a = f"centr_{centrality_variant}_ft"
ccol_b = f"centr_{centrality_variant}_tf"


In [ ]:
def summarize_ps(df, pscol):
    parts = pscol.split("_")
    t = 1
    ps_name = parts[-1]
    if "t" in parts[-1]:
        ps_name = parts[-2]
        t = parts[-1].lstrip("t")
    print("name:", ps_name, "t:", t)
    key = f"{ps_name} t{t}"
    
    # by % of ps value
    h0 = df[df[pscol] >= df[pscol].max()*0.2]
    h1 = df[df[pscol] >= df[pscol].max()*0.1]
    h2 = df[df[pscol] >= df[pscol].quantile(0.99)]
    #ax = h1.bikeability.hist(bins=[0.0,0.2,0.4,0.6,0.8,1.0], label=key, histtype="bar")
    #ax.legend()
    #plt.show()
    h0b = h0.bikeability
    h1b = h1.bikeability
    h2b = h2.bikeability
    h0_ba.append(h0b)
    h1_ba.append(h1b)
    h2_ba.append(h2b)
    keys.append(key)
    
    return {"ps": key, "aoi":aoi_name, 
            "h0_n": len(h0b), "h0_share": len(h0b)/len(df), "h0_b_mean": h0b.mean(), "h0_b_median": h0b.quantile(0.5),
            "h1_n": len(h1b), "h1_share": len(h1b)/len(df), "h1_b_mean": h1b.mean(), "h1_b_median": h1b.quantile(0.5),
            "h2_n": len(h2b), "h2_share": len(h2b)/len(df), "h2_b_mean": h2b.mean(), "h2_b_median": h2b.quantile(0.5)
            } #| h1["bikeability"].describe().to_dict()
    #return {"ps": key, "dta": df[pscol].describe()}

In [ ]:
stats = []
for aoi_name in aoi_names:
    cols = nh.read_cols_from_gpkg(file_ps.replace("<aoi_name>", aoi_name))
    pscols = []
    for c in cols:
        if c.find(centrality_variant) > -1 and (not (c.endswith("_ft") or c.endswith("_tf"))):
            pscols.append(c)
    generic_cols = []
    for c in cols:
        if not c.startswith("centr_") and not c.startswith("ps_"):
            generic_cols.append(c)
    # read file
    ps = gpd.read_file(file_ps.replace("<aoi_name>", aoi_name), columns=(generic_cols + pscols + [ccol_a, ccol_b]), mask=h.get_aoi_extent(aoi_name))
    ps.loc[:,"bikeability"] = ps[[f"index_{mode}_ft", f"index_{mode}_tf"]].mean(axis=1)
    ps.loc[:,"centrality"] = ps[[ccol_a, ccol_b]].mean(axis=1)
    ps.head()
    if len(ps) < 1:
        print(f"WARN: no segments received for the core area of '{aoi_name}'. Please check input. Skipping.")
        continue
    # prepare for processing
    h0_ba = []
    h1_ba = []
    h2_ba = []
    keys = []
    for pscol in pscols:
        stats.append(summarize_ps(ps, pscol))
    plt.hist(h0_ba, label=keys, bins=[0.0, 0.2, 0.4, 0.6, 0.8, 1.0])
    plt.legend()
    h.save_plot(f"ps_{centrality_variant}_h0_bikeability_hist", os.path.join("plots", aoi_name))
    plt.hist(h1_ba, label=keys, bins=[0.0, 0.2, 0.4, 0.6, 0.8, 1.0])
    plt.legend()
    h.save_plot(f"ps_{centrality_variant}_h1_bikeability_hist", os.path.join("plots", aoi_name))
    plt.hist(h2_ba, label=keys, bins=[0.0, 0.2, 0.4, 0.6, 0.8, 1.0])
    plt.legend()
    h.save_plot(f"ps_{centrality_variant}_h2_bikeability_hist", os.path.join("plots", aoi_name))
stats_df = pd.DataFrame(stats)
display(stats_df)

In [ ]:
g = stats_df.groupby("ps")
g.h1_b_mean.describe()

In [ ]:
g.h2_b_mean.describe()

In [ ]:
g.h0_b_mean.describe().loc[["p1 t1", "p2 t1", "p3 t1", "p1 t0.8", "p2 t0.8", "p3 t0.8"],["min", "mean", "max"]].plot()
h.save_plot(f"ps_{centrality_variant}_h0_mean_bikeability", "plots")

In [ ]:
g.h1_b_mean.describe().loc[["p1 t1", "p2 t1", "p3 t1", "p1 t0.8", "p2 t0.8", "p3 t0.8"],["min", "mean", "max"]].plot()
h.save_plot(f"ps_{centrality_variant}_h1_mean_bikeability", "plots")

In [ ]:
g.h2_b_mean.describe().loc[["p1 t1", "p2 t1", "p3 t1", "p1 t0.8", "p2 t0.8", "p3 t0.8"],["min", "mean", "max"]].plot()
h.save_plot(f"ps_{centrality_variant}_h2_mean_bikeability", "plots")

In [ ]:
stats_df.to_csv(os.path.join(dir_data, "ps_stats.csv"))